# Open Source News Analysis

By Ben Welsh and Scott Klein

This notebook demonstrates the findings presented at the News Product Alliance conference on Oct. 24, 2025. It analyzes the decline of open source news projects over the past decade using data from GitHub repositories linked to news organizations.

## Import Python and data

In [1]:
from pathlib import Path

import pandas as pd
import altair as alt
from rich import print

In [2]:
# Allow for Altair to use the Vega Fusion renderer for large datasets
alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [3]:
transform_path = Path(".") / "data" / "transformed"

In [4]:
org_repos_df = pd.read_csv(
    transform_path / "org-repos.csv",
    parse_dates=["created_at"],
)

In [5]:
org_repos_df.head(1)

,org,name,full_name,homepage,description,language,created_at,updated_at,pushed_at,stargazers_count,watchers_count,forks_count,open_issues_count,license,topics
0,abcnews,.github,abcnews/.github,NaN,ABC public Github profile,NaN,2023-01-31 03:49:26+00:00,2023-01-31 03:49:26+00:00,2023-02-02 03:37:20+00:00,0,0,0,0,NaN,[]


## Count the organizations and repositories

In [6]:
print(f"{len(org_repos_df):,} public repos")

14,624 public repos

In [7]:
print(f"Released by {org_repos_df.org.nunique()} news organizations")

Released by 309 news organizations

In [8]:
print(
    f"From {org_repos_df['created_at'].min().year} to {org_repos_df['created_at'].max().year}."
)

From 2008 to 2026.

## Analyze the trend over time

In [9]:
alt.Chart(
    org_repos_df, title="New repositories by year", width=600, height=400
).mark_bar().encode(
    x=alt.X("year(created_at):O", title=None),
    y=alt.Y("count():Q", title=None),
    tooltip=[
        alt.Tooltip("year(created_at):O", title="Year"),
        alt.Tooltip("count():Q", title="New repositories"),
    ],
)

alt.Chart(...)

In [10]:
# Group and count by created year
repos_by_year = (
    org_repos_df.assign(created_year=org_repos_df["created_at"].dt.year)
    .groupby("created_year")
    .size()
    .reset_index(name="repo_count")
)

In [11]:
peak_year = repos_by_year.loc[repos_by_year["repo_count"].idxmax()].repo_count
last_year = repos_by_year.loc[
    repos_by_year["created_year"] == 2024, "repo_count"
].values[0]

In [12]:
print(
    f"{abs((last_year - peak_year) / peak_year * 100):.0f}% decline in new repos from the peak year to last year."
)

83% decline in new repos from the peak year to last year.

In [13]:
repos_by_year.to_csv(transform_path / "repos-by-year.csv", index=False)

## Create a crosstab of organizations and repositories

In [14]:
# Count the number of new repos by year
org_repos_df["year"] = org_repos_df["created_at"].dt.year
org_counts = org_repos_df.groupby(["org", "year"]).size().reset_index(name="count")

In [15]:
# Save the counts to a CSV file
org_counts.to_csv(transform_path / "org-repos-by-year.csv", index=False)

In [16]:
org_counts.head(1)

,org,year,count
0,abcnews,2014,1


In [31]:
# Output a selected number of orgs for a mini-multiples chart on Datawrapper
selected_orgs = org_counts[
    org_counts.org.isin(
        [
            # Still big and doing well
            "guardian",
            "nytimes",
            "newsdev",
            "nytlabs",
            "propublica",
            # Out of business
            "micnews",
            "buzzfeednews",
            "fivethirtyeight",
        ]
    )
]

# Merge the newsdev and nytlabs and nytimes accounts since they are the same org
selected_orgs.loc[selected_orgs.org == "newsdev", "org"] = "nytimes"
selected_orgs.loc[selected_orgs.org == "nytlabs", "org"] = "nytimes"

# Regroup after the merge
selected_orgs = selected_orgs.groupby(["org", "year"]).sum().reset_index()

# Clean up the organization names
name_map = {
    "guardian": "The Guardian",
    "nytimes": "The New York Times",
    "propublica": "ProPublica",
    "micnews": "Mic",
    "buzzfeednews": "BuzzFeed News",
    "fivethirtyeight": "FiveThirtyEight",
}

selected_orgs["org"] = selected_orgs["org"].map(name_map)

# Ensure we have the right number of orgs
assert selected_orgs.org.nunique() == 6

# Pivot so it's one row per year and one column per org
selected_orgs_pivot = (
    selected_orgs.pivot(index="year", columns="org", values="count")
    .fillna(0)
    .reset_index()[
        [
            "year",
            "Mic",
            "BuzzFeed News",
            "FiveThirtyEight",
            "The New York Times",
            "The Guardian",
            "ProPublica",
        ]
    ]
)

In [32]:
selected_orgs_pivot.head(2)

org,year,Mic,BuzzFeed News,FiveThirtyEight,The New York Times,The Guardian,ProPublica
0,2009,0.0,0.0,0.0,0.0,0.0,2.0
1,2010,0.0,0.0,0.0,5.0,9.0,8.0


In [33]:
# Write out the selected orgs
selected_orgs_pivot.to_csv(
    transform_path / "selected-org-repos-by-year.csv", index=False
)

In [34]:
# Output a "good news" list of orgs for a mini-multiples chart on Datawrapper
goodnews_orgs = org_counts[
    org_counts.org.isin(["the-pudding", "bellingcat", "city-bureau"])
]

# Regroup after the merge
goodnews_orgs = goodnews_orgs.groupby(["org", "year"]).sum().reset_index()

# Clean up the organization names
name_map = {
    "the-pudding": "The Pudding",
    "bellingcat": "Bellingcat",
    "city-bureau": "City Bureau",
}

goodnews_orgs["org"] = goodnews_orgs["org"].map(name_map)

# Ensure we have the right number of orgs
assert goodnews_orgs.org.nunique() == 3

# Pivot so it's one row per year and one column per org
goodnews_orgs_pivot = (
    goodnews_orgs.pivot(index="year", columns="org", values="count")
    .fillna(0)
    .reset_index()[
        [
            "year",
            "The Pudding",
            "City Bureau",
            "Bellingcat",
        ]
    ]
)

# Fill back to 2008 with zeros
goodnews_orgs_pivot = pd.merge(
    pd.DataFrame({"year": range(2008, 2026)}),
    goodnews_orgs_pivot,
    on="year",
    how="outer",
).fillna(0)

In [21]:
goodnews_orgs_pivot.head(2)

,year,The Pudding,City Bureau,Bellingcat
0,2008,0.0,0.0,0.0
1,2009,0.0,0.0,0.0


In [22]:
goodnews_orgs_pivot.tail(2)

,year,The Pudding,City Bureau,Bellingcat
16,2024,31.0,11.0,8.0
17,2025,14.0,3.0,7.0


In [23]:
goodnews_orgs_pivot.to_csv(
    transform_path / "goodnews-org-repos-by-year.csv", index=False
)

In [24]:
# Pivot that to be a wide format
org_pivot = org_counts.pivot(index="org", columns="year", values="count").fillna(0)

In [25]:
# Calculate the total number of repos for each org
org_pivot["total"] = org_pivot.sum(axis=1)

In [26]:
# Calculate the percentage change from 2016, the peak year, to 2024, the most recent year
org_pivot["percent_change_16to24"] = (org_pivot[2024] - org_pivot[2016]) / org_pivot[
    2016
]

In [27]:
# Calculate some annual averages
org_pivot["annual_avg_14to23"] = org_pivot.loc[:, 2014:2023].mean(axis=1)
org_pivot["annual_avg_13to17"] = org_pivot.loc[:, 2013:2017].mean(axis=1)

In [28]:
# Go back to the org_df and calculate the most recent update_at time for each org
latest_update = (
    org_repos_df.groupby("org")["updated_at"]
    .max()
    .reset_index()
    .rename(columns={"updated_at": "latest_update"})
)

In [29]:
# Merge that on to the pivot
org_pivot = org_pivot.merge(latest_update, on="org", how="left")

In [30]:
# Write out the data
org_pivot.to_csv(transform_path / "org-activity.csv", index=False)